## 1. Initial Set-Up/Housekeeping to load needed modules/objects

In [3]:
!conda install -c anaconda lxml --yes
!conda install -c conda-forge tabulate --yes
!conda install -c anaconda beautifulsoup4 --yes


import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         154 KB  anaconda

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0  conda-forge --> 2019.9.11-py36_0 anaconda
    openssl: 1.1.1c-h516909a_0 conda-forge --> 1.1.1-h7b6447c_0 anaconda


certifi-2019.9.11    | 154 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 

## 2. Retrieve the data for Toronto and place into a dataframe

In [6]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df = df[0]
df.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 3. Exclude those rows with Borough of "Not assigned" and prin

In [7]:
df = df[df["Borough"]!="Not assigned"]
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## 4. Concatenate the Neighbourhood and/or reassign it to the Boruough

In [8]:
#df["NewNeighbourhood"] = df.groupby('Postcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
df = df.groupby('Postcode').agg({'Borough':'min','Neighbourhood':', '.join})
print(df[df['Neighbourhood'] == "Not assigned"]['Neighbourhood'].count())
df.loc[df['Neighbourhood'] == "Not assigned",'Neighbourhood'] = df['Borough']
print(df[df['Neighbourhood'] == "Not assigned"]['Neighbourhood'].count())
df = df.reset_index()
df.head()

1
0


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 5. Display the shape of the dataframe

In [9]:
df.shape

(103, 3)